In [1]:
import numpy as np
import glob
import os
import json
import random
import re

In [2]:
import spacy
import scispacy

from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_web_sm")

In [1]:
import nlpaug.augmenter.word as naw
aug = naw.BackTranslationAug()


In [7]:
s = set()
with open("train_HQS.txt", 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        for m in patientDict["1_medical"]:
            s.add(m)
        for m in patientDict["2_medical"]:
            s.add(m)

with open("validation_HQS.txt", 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        for m in patientDict["1_medical"]:
            s.add(m)
        for m in patientDict["2_medical"]:
            s.add(m)
            
with open("test_HQS.txt", 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        for m in patientDict["1_medical"]:
            s.add(m)
        for m in patientDict["2_medical"]:
            s.add(m)

ALL_medical_term = list(s)

In [8]:
len(ALL_medical_term)

1548

In [9]:
# positive set only 234

path_train, path_base, count, no_type1, no_type2 = "train_HQS.txt", "Positive/", 0, set(), set()
with open(path_train, 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        path_id = path_base + str(count) + "/"
        idx = count
        count += 1
        if not os.path.exists(path_id): os.makedirs(path_id)
            
        #1: the reference summary itself is truthful
        if len(patientDict["1_medical"]) == 0:
            with open(path_id + "positive1.txt", 'w') as test_file:
                test_file.write(patientDict["summary"].strip())
        else:
            no_type1.add(idx)
        
        #2: extract another utterance with using the last medical term (the last one in "2_medical")
        if len(patientDict["2_medical"]) > 0:
            target_term = patientDict["2_medical"][-1].lower()
            doc = nlp(patientDict["question"])
            sent_list = list(doc.sents)
            for i in range(len(sent_list)-1, -1, -1):
                if target_term in str(sent_list[i]).lower() or singularize(target_term) in str(sent_list[i]).lower():
                    with open(path_id + "positive2.txt", 'w') as test_file:
                        test_file.write(str(sent_list[i]).strip())
                    break
                #no_type2.add(idx)
        else:
            no_type2.add(idx)


        
#3: extract the longest sentence for any training instances without positive2
count = 0
with open(path_train, 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        idx = count
        path_id = path_base + str(count) + "/"
        count += 1
        if idx in no_type2:
            doc = nlp(patientDict["question"])
            sent_list = list(doc.sents)
            max_len, max_idx = 0, -1
            for i in range(len(sent_list)-1, -1, -1):
                if "[name]" in str(sent_list[i]).lower():
                    continue 
                if "[contact]" in str(sent_list[i]).lower():
                    continue
                if "[location]" in str(sent_list[i]).lower():
                    continue
                if len(str(sent_list[i]).strip()) >= max_len:
                    max_idx = i
                    max_len = len(str(sent_list[i]).strip())
            with open(path_id + "positive3.txt", 'w') as test_file:
                test_file.write(str(sent_list[max_idx]).strip())
        
        #4: if reference summary is unfaithful, machine translation to perform data augmentation on #2 or #3
        if idx in no_type1:
            target_eg = ""
            if idx not in no_type2:
                with open(path_id + "positive2.txt", 'r', encoding='utf8') as f:
                    target_eg = f.readlines()[0]
            else:
                with open(path_id + "positive3.txt", 'r', encoding='utf8') as f:
                    target_eg = f.readlines()[0]
            if target_eg == "":
                print("empty target:", idx)
                
            augmented_data = aug.augment(target_eg)[0]
            with open(path_id + "positive4.txt", 'w') as test_file:
                test_file.write(augmented_data.strip())
            
                

In [10]:
# negative set only 234

path_train, path_base, count = "train_HQS.txt", "Negative/", 0
with open(path_train, 'r') as input:
    for jsonObj in input:
        patientDict = json.loads(jsonObj)
        path_id = path_base + str(count) + "/"
        if not os.path.exists(path_id): os.makedirs(path_id)
        
        #1: the reference summary itself is untruthful
        if len(patientDict["1_medical"]) > 0:
            with open(path_id + "negative1.txt", 'w') as test_file:
                test_file.write(patientDict["summary"].strip())
                
        #2: change 1 truthful medical term (the 1st one in "2_medical") to another untruthful one
        if len(patientDict["2_medical"]) + len(patientDict["1_medical"]) > 0:
            truthful_medical_list, random_idx = patientDict["2_medical"] + patientDict["1_medical"], random.randint(0, 1547)
            while ALL_medical_term[random_idx] in truthful_medical_list:
                random_idx = random.randint(0, 1547)

            modified_summary = patientDict["summary"].replace(truthful_medical_list[0], ALL_medical_term[random_idx])
            with open(path_id + "negative2.txt", 'w') as test_file:
                test_file.write(modified_summary)
        
        #3: append an untruthful medical term to the start of the reference summary
        truthful_medical_list, random_idx = patientDict["2_medical"] + patientDict["1_medical"], random.randint(0, 1547)
        while ALL_medical_term[random_idx] in truthful_medical_list:
            random_idx = random.randint(0, 1547)
        modified_summary = ALL_medical_term[random_idx] + " " + patientDict["summary"]
        with open(path_id + "negative3.txt", 'w') as test_file:
            test_file.write(modified_summary)
            
        #4: append an untruthful medical term to the END of the reference summary
        prev_idx = random_idx
        random_idx = random.randint(0, 1547)
        while prev_idx == random_idx or ALL_medical_term[random_idx] in truthful_medical_list:
            random_idx = random.randint(0, 1547)
        modified_summary = patientDict["summary"] + " " + ALL_medical_term[random_idx]
        with open(path_id + "negative4.txt", 'w') as test_file:
            test_file.write(modified_summary)
            
        #5: change numerical values
        arabic = re.findall(r'\d+', patientDict["summary"])
        if len(arabic) > 0:
            modified_summary = patientDict["summary"]
            for a in arabic:
                random_idx = random.randint(20, 30)
                while random_idx == int(a):
                    random_idx = random.randint(20, 30)
                modified_summary = modified_summary.replace(a, str(random_idx))
            
            with open(path_id + "negative5.txt", 'w') as test_file:
                test_file.write(modified_summary)
            
            
        
        #6: Swap the entities using NER
        ent_list = list(nlp(patientDict["summary"]).ents)
        if len(ent_list) >= 2:
            modified_summary = patientDict["summary"]
            for i in range(len(ent_list)):
                modified_summary = modified_summary.replace(str(ent_list[i]), str(i))
            
            num = 0
            for i in range(len(ent_list)-1, -1, -1):
                modified_summary = modified_summary.replace(str(num), str(ent_list[i]))
                num += 1
            
            with open(path_id + "negative6.txt", 'w') as test_file:
                test_file.write(modified_summary)
            
        
        count += 1